# 🐍 Navigator and Websocket for Python! 🚀

In this interactive guide, you'll learn how to use Navigator WebAssistant.

Navigator WebAssistant is webserver that handle navigator-rs library and allows users to use it through web based applications.

## What You'll Learn

With this guide, you will learn how to:

1. 🌟 Initiate your Navigator-WebAssistant
2. 🎉 Use websocket to connect to webassistant's service
3. 🌡️ Plot live data
4. ⚙️ Send commands

Click bellow to start it!

## 1. Initiate your Navigator-Assistant

The easiest way to start using Navigator-WebAssistant, is through itself BlueOS's Extension.

You can find it on:
http://blueos.local/tools/extensions-manager

Or you can download the navigator-web-assistant binary and run locally.

## 2. Use websocket to connect to assistant's service

Now with this first example you will be abble to connect to navigator-assistant and receive it's first websocket message.

Please install the bellow package to use websockets.

In [ ]:
pip install --user websocket-client

Now you can run and check your first websocket message from Navigator-Assistant!

In [ ]:
import websocket

def on_message(ws, message):
    print(message)
    wsapp.close()
def on_error(wsapp, err):
    print("Got a an error: ", err)

# Change ip adress and port to yours Navigator Assitant Extension OR self running Navigator service
wsapp = websocket.WebSocketApp("ws://192.168.15.200:32769/ws",

on_message = on_message,
on_error=on_error)
wsapp.run_forever()  

## 3. Plot live data

Now with this next example you will be abble to plot live data.
The follow example is like the Navigator, but we are updating the temperature values as we receive websocket broadcast messages.

Please change the stop_on_max value if you want it to continue after max value.

In [ ]:
import websocket
import json
import plotly.graph_objs as go
from IPython.display import display, clear_output

# Initialize empty lists to store the data for the graph
max_data_points = 10
timestamps = []
temperatures = []

# Change this to have 
stop_on_max= True

# WebSocket message handler
def on_message(ws, message):
    data = json.loads(message)
    if 'input' in data and data['input']:
        for entry in data['input']:
            if entry['type'] == 'Temperature':
                
                timestamps.append(data['timestamp'])
                temperatures.append(entry['value'])

                if len(timestamps) > max_data_points:
                    timestamps.pop(0)
                    temperatures.pop(0)
                    if stop_on_max : ws.close()

                # Create a Plotly figure
                figure = go.Figure()
                
                # Set layout options
                figure.update_layout(
                    title='Real-time Temperature Graph',
                    xaxis_title='Timestamp',
                    yaxis_title='Temperature (°C)'
                )

                figure.add_trace(go.Scatter(x=timestamps, y=temperatures, mode='lines+markers'))
                
                # Clear the output to update the graph in the Jupyter Notebook
                clear_output(wait=True)
                display(figure)

# WebSocket connection
ws = websocket.WebSocketApp("ws://192.168.15.200:32769/ws",
                            on_message=on_message)

# Start WebSocket connection
ws.run_forever()


## 4. Send commands

Now this example use a different aproach to handle the websocket connection.

Notice that we used a regex filter, the websocket connection allows this.

So, websocket will broadcast only messages containing "output" on it's body.

In [ ]:
from websocket import create_connection
import time

# Change the IP address and port to your WebSocket server
ws = create_connection("ws://192.168.15.200:32769/ws?filter=output")

# Send a message to the WebSocket server
message_to_send = '/output/userled/{ "userled": "Led1", "value": true }'
ws.send(message_to_send)

print("Every webscoket request receive this message:\n")
print(ws.recv())
print("\nFollowed by the broadcasted request:\n")
print(ws.recv())


print("\nNow it will change the userled states for 1 minute.")

# LED names and true/false values to iterate through
leds = ["Led1", "Led2", "Led3"]
states = [True, False]

# Start time
start_time = time.time()

# Run for 1 minute (60 seconds)
while time.time() - start_time < 60:
    # Iterate through LED names and states
    for led in leds:
        for state in states:
            message_to_send = f"/output/userled/{led}/{str(state).lower()}"
            ws.send(message_to_send)
            time.sleep(0.5)  #
        
ws.close()